In [2]:
from ultralytics import YOLO
from PIL import Image
from numpy import asarray
import cv2
import os
import tensorflow
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [3]:
def crop_and_save_objects(image_path,model_path,obj_model_path):
    
    obj_model = YOLO(obj_model_path)
    
    results=obj_model.predict(source=image_path,save=True,save_crop=True,device='cpu')
    for result in results:
        boxes = result.boxes.xyxy
    
    np_arr = boxes.detach().cpu().numpy()

    L=[]
    
    for i in np_arr:
        L.append(i)
    
    for i in range(0,len(L)):
        for j in range(0,len(L[i])): 
                L[i][j]=int(L[i][j])
    
    bounding_boxes=L
    
    # Load the model
    model = tf.keras.models.load_model(model_path, compile=False)
    model.compile(optimizer=tensorflow.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    # Load the image
    img = cv2.imread(image_path)

    # Create a directory to store the cropped images
    output_dir = f"{os.path.splitext(image_path)[0]}_cropped"
    os.makedirs(output_dir, exist_ok=True)

    # Create a DataFrame to store the purity summary for each object
    summary_df = pd.DataFrame(columns=["Object", "Purity"])

    # Loop through each bounding box and crop the object within it
    for i, bbox in enumerate(bounding_boxes):
        # Extract the coordinates of the bounding box
        x1, y1, x2, y2 = map(int, bbox)

        # Crop the object within the bounding box
        cropped_img = img[y1:y2, x1:x2]

        # Resize the cropped image to 224x224
        resized_img = cv2.resize(cropped_img, (224, 224))

        # Convert the resized image to a NumPy array
        img_array = np.asarray(resized_img)

        # Normalize the pixel values to be in the range [0, 1]
        img_array = img_array / 255.0

        # Make the prediction using the model
        pred = model.predict(np.expand_dims(img_array, axis=0))[0]

        # Determine if the object is pure or impure based on the prediction
        if pred > 0.5:
            label = "pure"
        else:
            label = "impure"

        # Save the cropped image as a NumPy array and label to the output directory with a unique name
        output_path = os.path.join(output_dir, f"object_{i}_{label}.npy")
        np.save(output_path, img_array)

        # Add the purity label to the summary DataFrame
        summary_df.loc[i] = [f"object_{i}", label]

    # Create a table summarizing the purity of each cropped image
    table_df = pd.pivot_table(summary_df, index=["Object"], columns=["Purity"], aggfunc=len, fill_value=0)
    table_df.columns.name = None
    table_df.index.name = None

    # Print the summary table
    #print(table_df)

    # Calculate the sum of each class
    impure_sum = summary_df["Purity"].value_counts().get("impure", 0)
    pure_sum = summary_df["Purity"].value_counts().get("pure", 0)
    total_sum = impure_sum + pure_sum
    print("The Total number of Impure grains is ",impure_sum)
    print("The Total number of Pure grains is ",pure_sum)
    print("The Total number of Grains is ",total_sum)

    # Return the sum of each class
    #return impure_sum, pure_sum, total_sum


In [4]:
image_path = "Worst.jpg"
obj_model_path="best.pt"
model_path='C:/Users/Rohit-L490/EffNetB0_tf.kr.app_tr70val20test10_nopretrain.h5'

In [7]:
%timeit
crop_and_save_objects(image_path,model_path,obj_model_path)

Ultralytics YOLOv8.0.29  Python-3.9.13 torch-1.13.1+cpu CPU
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 C:\Users\Rohit-L490\Worst.JPG: 640x640 17 Corns, 241.3ms
Speed: 1.0ms pre-process, 241.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict19


1/1 [==============================] - 0s 100ms/step
The Total number of Impure grains is  17
The Total number of Pure grains is  0
The Total number of Grains is  17
